## Evidence Pipeline Test Data Generator

Generates input data for evidence preparation pipeline testing

Process:
- choose genes to extract data for
- load raw evidence objects (subset of downloads from GS)
- load gene objects dumped from ES (extract/gene.json)
- subset evidence and genes to target list
- save the evidence records and the corresponding gene objects

In [1]:
import $file.^.testgeninit, testgeninit._
import $file.^.sparkinit, sparkinit._
import $file.^.pathinit, pathinit._
import $file.^.cpinit, cpinit._
import ss.implicits._
import org.apache.spark.sql.functions._
import java.nio.file.Paths

Loading spark-stubs
Creating SparkSession


Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
19/12/10 10:07:11 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


import $file.$            , testgeninit._

import $file.$          , sparkinit._

import $file.$         , pathinit._

import $file.$       , cpinit._

import ss.implicits._

In [2]:
// Initialize all parameters utilitzed
val extractDir = EXTRACT_DIR
val testInputDir = TEST_PIPELINE_DIR.resolve("input")
val genes = GENE_SET_1 // from testgeninit

extractDir: java.nio.file.Path = /home/eczech/data/ot/extract
testInputDir: java.nio.file.Path = /home/eczech/repos/ot-scoring/src/test/resources/pipeline_test/input
genes: List[String] = List(
  "ENSG00000141510",
  "ENSG00000169174",
  "ENSG00000105397"
)

### Load

In [3]:
// Load gene data ES extract
val dfg = ss.read.json(extractDir.resolve("gene.json").toString)
    .filter($"ensembl_gene_id".isin(genes: _*))
dfg.count()

json at cmd2.sc:1

16 / 16

19/12/10 10:07:24 WARN Utils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.debug.maxToStringFields' in SparkEnv.conf.


count at cmd2.sc:3

16 / 16

count at cmd2.sc:3

1 / 1

dfg: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [_private: struct<facets: struct<reactome: struct<pathway_code: array<string>, pathway_type_code: array<string>>>, suggestions: struct<input: array<string>, output: string ... 1 more field>>, alias_name: array<string> ... 60 more fields]
res2_1: Long = 3L

In [5]:
// Load raw evidence extract 
val dfe = ss.read.json(extractDir.resolve("evidence_raw.json").toString)
    .filter(element_at(split($"target.id", "/"), -1).isin(genes: _*))
dfe.count()

json at cmd4.sc:1

9 / 9

count at cmd4.sc:3

9 / 9

count at cmd4.sc:3

1 / 1

dfe: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [_corrupt_record: string, access_level: string ... 10 more fields]
res4_1: Long = 7621L

### Export

In [7]:
var path = testInputDir.resolve("gene_ese.json").toString
dfg.coalesce(1).write
    .format("json")
    .mode("overwrite")
    .option("compression", "gzip")
    .save(path)
println(s"Saved ${dfg.count()} gene records to '$path'")

save at cmd6.sc:8

1 / 1

count at cmd6.sc:9

16 / 16

count at cmd6.sc:9

1 / 1

Saved 3 gene records to '/home/eczech/repos/ot-scoring/src/test/resources/pipeline_test/input/gene_ese.json'


path: String = "/home/eczech/repos/ot-scoring/src/test/resources/pipeline_test/input/gene_ese.json"

In [8]:
var path = testInputDir.resolve("evidence_raw.json").toString
dfe.coalesce(1).write
    .format("json")
    .mode("overwrite")
    .option("compression", "gzip")
    .save(path)
println(s"Saved ${dfe.count()} evidence records to '$path'")

save at cmd7.sc:8

1 / 1

count at cmd7.sc:9

9 / 9

count at cmd7.sc:9

1 / 1

Saved 7621 evidence records to '/home/eczech/repos/ot-scoring/src/test/resources/pipeline_test/input/evidence_raw.json'


path: String = "/home/eczech/repos/ot-scoring/src/test/resources/pipeline_test/input/evidence_raw.json"